# Ground-roll attenuation model with attention

Raw field seismograms contain noises of various nature. One type of noise is assosiated with ground-roll waves propagation. The model should suppress the ground-roll noise and keep desired signal unchanged.

In [2]:
import os
import sys
import shutil

from tqdm import tqdm_notebook
import numpy as np

sys.path.append('../../..')

from seismicpro.batchflow import Pipeline, B, L, I

from seismicpro.src import SeismicDataset, TraceIndex, merge_segy_files
from seismicpro.models import UnetAtt 
from seismicpro.models.unetatt_3 import UnetAttention4, UnetAttention3

%env CUDA_VISIBLE_DEVICES=5

env: CUDA_VISIBLE_DEVICES=5


In [3]:
def exp_stack(x):
    return np.expand_dims(np.vstack(x), -1)

def make_data_inference(batch, **kwagrs):
    return {"feed_dict": {'trace_raw': exp_stack(batch.raw),
                          'offset': np.vstack(batch.trace_headers('offset'))}}

## Inference

For inference we need a ```TraceIndex```, we also keep all SEGY headers from original traces:

In [4]:
def inference(model, model_path, path_raw, output_path=None, tmp_dump_path='tmp', clear=False):
    if os.path.exists(tmp_dump_path):
        shutil.rmtree(tmp_dump_path)
    
    os.makedirs(tmp_dump_path)
    
    inference_index = TraceIndex(name='raw', extra_headers='all', path=path_raw)
    inference_ppl = (SeismicDataset(inference_index).p
                     .load_model("static", model, 'unet', path=model_path)
                     .init_variable('res')
                     .load(components='raw', fmt='segy', tslice=np.arange(3000))
                     .predict_model('unet', make_data=make_data_inference,
                                    fetches=['out_lift'], save_to=B('raw'))
                     .dump(path=L(lambda x: os.path.join(tmp_dump_path, str(x) + '.sgy'))(I()),
                           src='raw', fmt='segy', split=False)
                 )
    inference_ppl.run(2000, n_epochs=1, drop_last=False, shuffle=False, bar=True)
    
    if output_path is None:
        clear = False
        output_path = os.path.join(tmp_dump_path, 'merged.sgy')
        
    merge_segy_files(output_path=output_path, extra_headers='all', path=os.path.join(tmp_dump_path, '*.sgy'))
    
    if clear:
        if os.path.exists(tmp_dump_path):
            shutil.rmtree(tmp_dump_path)        
    
    return output_path

In [4]:
%%time

path_raw = '/notebooks/data/noise_dataset_1/DN02A_LIFT_AMPSCAL_norm2.sgy'

paths = {
    UnetAttention3: "./saved_models/noise_dataset_1/UnetAttention3/2019-10-08_13:01:12.296337+03:00",
    UnetAttention4: "./saved_models/noise_dataset_1/UnetAttention4/2019-10-08_13:27:57.070603+03:00",
    UnetAtt: "./saved_models/noise_dataset_1/UnetAtt/2019-10-08_13:15:31.021212+03:00",
}

for model, model_path in paths.items():
    output_path = inference(model, model_path, path_raw, output_path=os.path.join(model_path, 'noise_dataset_1_out.sgy'), clear=True)
    print(output_path)

INFO:tensorflow:Restoring parameters from ./saved_models/noise_dataset_1/UnetAttention4/2019-10-08_13:27:57.070603+03:00/model-2700


100%|██████████| 88/88 [00:42<00:00,  2.14it/s]


./saved_models/noise_dataset_1/UnetAttention4/2019-10-08_13:27:57.070603+03:00/noise_dataset_1_out.sgy
INFO:tensorflow:Restoring parameters from ./saved_models/noise_dataset_1/UnetAttention3/2019-10-08_13:01:12.296337+03:00/model-2700


100%|██████████| 88/88 [00:42<00:00,  2.08it/s]


./saved_models/noise_dataset_1/UnetAttention3/2019-10-08_13:01:12.296337+03:00/noise_dataset_1_out.sgy
INFO:tensorflow:Restoring parameters from ./saved_models/noise_dataset_1/UnetAtt/2019-10-08_13:15:31.021212+03:00/model-2700


100%|██████████| 88/88 [00:43<00:00,  2.08it/s]


./saved_models/noise_dataset_1/UnetAtt/2019-10-08_13:15:31.021212+03:00/noise_dataset_1_out.sgy
CPU times: user 6min 26s, sys: 1min 25s, total: 7min 51s
Wall time: 13min 59s


In [9]:
%%time

path_raw = '/notebooks/data/YL1_UKM/01_NA-gr_input_prep_norm2.sgy'

paths = {
    UnetAttention3: "./saved_models/noise_dataset_1/UnetAttention3/2019-10-08_13:01:12.296337+03:00",
    UnetAttention4: "./saved_models/noise_dataset_1/UnetAttention4/2019-10-08_13:27:57.070603+03:00",
    UnetAtt: "./saved_models/noise_dataset_1/UnetAtt/2019-10-08_13:15:31.021212+03:00",
}

for model, model_path in paths.items():
    output_path = inference(model, model_path, path_raw, output_path=os.path.join(model_path, 'YL1_UKM_out.sgy'), clear=True)
    print(output_path)


INFO:tensorflow:Restoring parameters from ./saved_models/noise_dataset_1/UnetAttention4/2019-10-08_13:27:57.070603+03:00/model-2700


100%|██████████| 147/147 [01:08<00:00,  2.00it/s]


./saved_models/noise_dataset_1/UnetAttention4/2019-10-08_13:27:57.070603+03:00/YL1_UKM_out.sgy
INFO:tensorflow:Restoring parameters from ./saved_models/noise_dataset_1/UnetAttention3/2019-10-08_13:01:12.296337+03:00/model-2700


100%|██████████| 147/147 [01:09<00:00,  2.12it/s]


./saved_models/noise_dataset_1/UnetAttention3/2019-10-08_13:01:12.296337+03:00/YL1_UKM_out.sgy
INFO:tensorflow:Restoring parameters from ./saved_models/noise_dataset_1/UnetAtt/2019-10-08_13:15:31.021212+03:00/model-2700


100%|██████████| 147/147 [01:08<00:00,  2.14it/s]


./saved_models/noise_dataset_1/UnetAtt/2019-10-08_13:15:31.021212+03:00/YL1_UKM_out.sgy
CPU times: user 10min 9s, sys: 1min 48s, total: 11min 57s
Wall time: 22min 34s


In [10]:
%%time

path_raw = '/notebooks/data/noise_dataset_1/DN02A_LIFT_AMPSCAL_norm2.sgy'

paths = {
    UnetAttention3: "./saved_models/YL1_UKM/UnetAttention3/2019-10-08_19:22:03.834257+03:00",
    UnetAttention4: "./saved_models/YL1_UKM/UnetAttention4/2019-10-08_17:51:27.015153+03:00",
    UnetAtt: "./saved_models/YL1_UKM/UnetAtt/2019-10-08_17:29:10.283045+03:00",
}

for model, model_path in paths.items():
    output_path = inference(model, model_path, path_raw, output_path=os.path.join(model_path, 'noise_dataset_1_out.sgy'), clear=True)
    print(output_path)

INFO:tensorflow:Restoring parameters from ./saved_models/YL1_UKM/UnetAttention4/2019-10-08_17:51:27.015153+03:00/model-4775


100%|██████████| 88/88 [00:43<00:00,  2.05it/s]


./saved_models/YL1_UKM/UnetAttention4/2019-10-08_17:51:27.015153+03:00/noise_dataset_1_out.sgy
INFO:tensorflow:Restoring parameters from ./saved_models/YL1_UKM/UnetAttention3/2019-10-08_19:22:03.834257+03:00/model-4775


100%|██████████| 88/88 [00:43<00:00,  2.15it/s]


./saved_models/YL1_UKM/UnetAttention3/2019-10-08_19:22:03.834257+03:00/noise_dataset_1_out.sgy
INFO:tensorflow:Restoring parameters from ./saved_models/YL1_UKM/UnetAtt/2019-10-08_17:29:10.283045+03:00/model-4775


100%|██████████| 88/88 [00:42<00:00,  2.18it/s]


./saved_models/YL1_UKM/UnetAtt/2019-10-08_17:29:10.283045+03:00/noise_dataset_1_out.sgy
CPU times: user 6min 18s, sys: 1min 6s, total: 7min 24s
Wall time: 13min 46s


In [11]:
%%time

path_raw = '/notebooks/data/YL1_UKM/01_NA-gr_input_prep_norm2.sgy'

paths = {
    UnetAttention3: "./saved_models/YL1_UKM/UnetAttention3/2019-10-08_19:22:03.834257+03:00",
    UnetAttention4: "./saved_models/YL1_UKM/UnetAttention4/2019-10-08_17:51:27.015153+03:00",
    UnetAtt: "./saved_models/YL1_UKM/UnetAtt/2019-10-08_17:29:10.283045+03:00",
}

for model, model_path in paths.items():
    output_path = inference(model, model_path, path_raw, output_path=os.path.join(model_path, 'YL1_UKM_out.sgy'), clear=True)
    print(output_path)


INFO:tensorflow:Restoring parameters from ./saved_models/YL1_UKM/UnetAttention4/2019-10-08_17:51:27.015153+03:00/model-4775


100%|██████████| 147/147 [01:14<00:00,  2.00it/s]


./saved_models/YL1_UKM/UnetAttention4/2019-10-08_17:51:27.015153+03:00/YL1_UKM_out.sgy
INFO:tensorflow:Restoring parameters from ./saved_models/YL1_UKM/UnetAttention3/2019-10-08_19:22:03.834257+03:00/model-4775


100%|██████████| 147/147 [01:11<00:00,  1.97it/s]


./saved_models/YL1_UKM/UnetAttention3/2019-10-08_19:22:03.834257+03:00/YL1_UKM_out.sgy
INFO:tensorflow:Restoring parameters from ./saved_models/YL1_UKM/UnetAtt/2019-10-08_17:29:10.283045+03:00/model-4775


100%|██████████| 147/147 [01:15<00:00,  1.55it/s]


./saved_models/YL1_UKM/UnetAtt/2019-10-08_17:29:10.283045+03:00/YL1_UKM_out.sgy
CPU times: user 10min 28s, sys: 1min 52s, total: 12min 21s
Wall time: 22min 50s


In [6]:
%%time

from itertools import product

path_raw_list = ['/notebooks/data/noise_dataset_1/DN02A_LIFT_AMPSCAL_norm2.sgy',
                '/notebooks/data/YL1_UKM/01_NA-gr_input_prep_norm2.sgy']
model_path_list = ['./saved_models/noise_dataset_1/UnetAttention2/2019-10-11_14:54:28.544402+03:00',
                  './saved_models/YL1_UKM/UnetAttention2/2019-10-11_15:28:25.671905+03:00']

for path_raw, model_path in product(path_raw_list, model_path_list):
    output_path = os.path.join(model_path, 'noise_dataset_1_out.sgy' if 'noise_dataset_1' in path_raw else 'YL1_UKM_out.sgy')
    output_path = inference(UnetAttention3, model_path, path_raw, output_path=output_path, clear=True)
    print(output_path)


INFO:tensorflow:Restoring parameters from ./saved_models/noise_dataset_1/UnetAttention2/2019-10-11_14:54:28.544402+03:00/model-2700


100%|██████████| 88/88 [00:42<00:00,  1.98it/s]


./saved_models/noise_dataset_1/UnetAttention2/2019-10-11_14:54:28.544402+03:00/noise_dataset_1_out.sgy
INFO:tensorflow:Restoring parameters from ./saved_models/YL1_UKM/UnetAttention2/2019-10-11_15:28:25.671905+03:00/model-4775


100%|██████████| 88/88 [00:41<00:00,  2.04it/s]


./saved_models/YL1_UKM/UnetAttention2/2019-10-11_15:28:25.671905+03:00/noise_dataset_1_out.sgy
INFO:tensorflow:Restoring parameters from ./saved_models/noise_dataset_1/UnetAttention2/2019-10-11_14:54:28.544402+03:00/model-2700


100%|██████████| 147/147 [01:12<00:00,  1.53it/s]


./saved_models/noise_dataset_1/UnetAttention2/2019-10-11_14:54:28.544402+03:00/YL1_UKM_out.sgy
INFO:tensorflow:Restoring parameters from ./saved_models/YL1_UKM/UnetAttention2/2019-10-11_15:28:25.671905+03:00/model-4775


100%|██████████| 147/147 [01:10<00:00,  1.94it/s]


./saved_models/YL1_UKM/UnetAttention2/2019-10-11_15:28:25.671905+03:00/YL1_UKM_out.sgy
CPU times: user 11min 6s, sys: 2min 4s, total: 13min 10s
Wall time: 23min
